In [24]:
import requests
import json
import pandas as pd
host = "https://starknet-mainnet.public.blastapi.io/rpc/v0_7"
JUNO_RPC_URL = host
stark_address = "0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d"

payload = {
    "jsonrpc": "2.0",
    "method": "starknet_getEvents",
    "params": {
        "filter": {
            "from_block": "latest",
            "to_block": "latest",
            "address": stark_address,
            "keys": [
                ["0x99cd8bde557814842a3121e8ddfd433a539b8c9f14bf31ebf108d12e6196e9"]
            ],
            "chunk_size": 2
        }
    },
    "id": 1
}

all_events = []
response = requests.post(JUNO_RPC_URL, json=payload)
data = response.json()
result = data.get("result", {})
all_events.extend(result.get("events", []))
continuation_token = result.get("continuation_token")

while continuation_token:
    payload["params"]["filter"]["continuation_token"] = continuation_token
    payload["id"] += 1
    response = requests.post(JUNO_RPC_URL, json=payload)
    data = response.json()
    result = data.get("result", {})
    all_events.extend(result.get("events", []))
    continuation_token = result.get("continuation_token")
formatted_events = []
for event in all_events:
    d = event.get("data", [])
    if len(d) >= 3:
        formatted_events.append({
            "from": d[0],
            "to": d[1],
            "value": float(int(d[2], 16) / (10 ** 18) ),
            "block_number": event.get("block_number"),
            "block_hash": event.get("block_hash"),
            "transaction_hash": event.get("transaction_hash")
        })

df = pd.DataFrame(formatted_events)
print(json.dumps({"events": formatted_events}, indent=2))

{
  "events": [
    {
      "from": "0x34d2a751fb800f42106cb1b05e90ac50a912c0737f03125c6ae0f5bc090a98e",
      "to": "0x1176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8",
      "value": 0.001745245242785654,
      "block_number": 1135782,
      "block_hash": "0x5db5359b08e4acf680564e8522ffb744516cc2f6661555cecd9300653a743d8",
      "transaction_hash": "0x4f29e4e2801ac86445cd64dda3f1a3b2e4875fa9693ae2406e1844bc4bb9b1a"
    },
    {
      "from": "0x387f3eb1d98632fbe3440a9f1385aec9d87b6172491d3dd81f1c35a7c61048f",
      "to": "0xd7a82eb09af5b96421954a749e57c991317b4449637028baa16fef19abae6e",
      "value": 2.149582312561705,
      "block_number": 1135782,
      "block_hash": "0x5db5359b08e4acf680564e8522ffb744516cc2f6661555cecd9300653a743d8",
      "transaction_hash": "0x70c5d9e88c1985e6c721db0e0f633df9495fcbfa45660840ecdf12937baed6e"
    },
    {
      "from": "0xd7a82eb09af5b96421954a749e57c991317b4449637028baa16fef19abae6e",
      "to": "0x1176a1bd84444c89232ec27754698

In [36]:
df2 = df[df['from'] == "0x34d2a751fb800f42106cb1b05e90ac50a912c0737f03125c6ae0f5bc090a98e"]
df.head()


,from,to,value,block_number,block_hash,transaction_hash
0,0x34d2a751fb800f42106cb1b05e90ac50a912c0737f03125c6ae0f5bc090a98e,0x1176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8,0.001745,1135782,0x5db5359b08e4acf680564e8522ffb744516cc2f6661555cecd9300653a743d8,0x4f29e4e2801ac86445cd64dda3f1a3b2e4875fa9693ae2406e1844bc4bb9b1a
1,0x387f3eb1d98632fbe3440a9f1385aec9d87b6172491d3dd81f1c35a7c61048f,0xd7a82eb09af5b96421954a749e57c991317b4449637028baa16fef19abae6e,2.149582,1135782,0x5db5359b08e4acf680564e8522ffb744516cc2f6661555cecd9300653a743d8,0x70c5d9e88c1985e6c721db0e0f633df9495fcbfa45660840ecdf12937baed6e
2,0xd7a82eb09af5b96421954a749e57c991317b4449637028baa16fef19abae6e,0x1176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8,0.002653,1135782,0x5db5359b08e4acf680564e8522ffb744516cc2f6661555cecd9300653a743d8,0x70c5d9e88c1985e6c721db0e0f633df9495fcbfa45660840ecdf12937baed6e
3,0x1b156fbf2fcad9612732ad2685d5ea0ff3bf2235ac47991ec7f292f46b7fcb8,0x1176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8,0.003225,1135782,0x5db5359b08e4acf680564e8522ffb744516cc2f6661555cecd9300653a743d8,0x1c5c11c141614d996d01e89f112a94dc7a4ef796be084c746442a2b3ecbf2c9
4,0x57cc63f124d30571c0d7f9f86fd417e977618db130a4c801d5c56abdffdbb4,0x40f0a2d940fc7624abadaf06c26d3e1249e0fd4b3fcccf88db56a4b628b23f3,3010.000000,1135782,0x5db5359b08e4acf680564e8522ffb744516cc2f6661555cecd9300653a743d8,0x3b3d3e7fe228c1363e7687f682aeccaacd0de7add281b1ff8a8125825ba031b
